# Accessing OpenAI through Vocareum API keys

In [1]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
os.getenv("OPENAI_API_KEY")

'voc-8957811371266772085699678d66e1475c97.62448043'

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key="voc-8957811371266772085699678d66e1475c97.62448043",
    base_url="https://openai.vocareum.com/v1"
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": "Talk like a pirate."},
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        },
    ],
)

print(completion.choices[0].message.content)

Arrr, to be seein' if a Python object be an instance of a particular class, ye use the trusty `isinstance()` function. Here be how ye do it:

```python
if isinstance(your_object, YourClass):
    print("Aye, the object be an instance of the class!")
else:
    print("Nay, the object be not of that class!")
```

Just replace `your_object` with the object ye be checkin', and `YourClass` with the name of the class ye be inspectin'. Arrr, that be the way to find the truth on the high seas of code!


In [4]:
from openai import OpenAI
client = OpenAI(
    base_url = openai.api_base,
    api_key = openai.api_key
)

In [4]:
def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


# Extracting data from Wikipedia

## Getting a list of all pages from a Wikipedia category

In [13]:
import wikipediaapi
import openai
import pandas as pd
import json
import os

In [17]:
def fetch_wikipedia_page(page_title):
    page = wiki_wiki.page(page_title)
    if page.exists():
        return page.text
    else:
        return None

In [18]:
wiki_wiki = wikipediaapi.Wikipedia('CustomChatbot (gabrielgoncalvesbr@gmail.com)', 'en')

In [19]:
if os.path.exists("nba_playoffs_wikipedia.json"):
    # Read the JSON file and load it into the dictionary
    with open("nba_playoffs_wikipedia.json", "r", encoding="utf-8") as json_file:
        dict_nba_playoffs_pages = json.load(json_file)
else:
    dict_nba_playoffs_pages = {}
    
    cat = wiki_wiki.page("Category:NBA playoffs")
    for title in cat.categorymembers:
        print(title)
        page_result = fetch_wikipedia_page(title)
        if page_result:
            dict_nba_playoffs_pages[title] = page_result

    with open("nba_playoffs_wikipedia.json", "w", encoding="utf-8") as json_file:
        json.dump(dict_nba_playoffs_pages, json_file, ensure_ascii=False, indent=4)


# Preparing the dataset

In [20]:
df_playoffs = pd.DataFrame.from_dict(
    dict_nba_playoffs_pages, 
    orient='index', 
    columns=['text']#, 'text']
).reset_index().rename(columns={'index': 'title'})
df_playoffs.head()

,title,text
0,NBA playoffs,The NBA playoffs is the annual elimination tou...
1,1950 NBA playoffs,The 1950 NBA playoffs was the postseason tourn...
2,1951 NBA playoffs,The 1951 NBA playoffs was the postseason tourn...
3,1952 NBA playoffs,The 1952 NBA playoffs was the postseason tourn...
4,1953 NBA playoffs,The 1953 NBA playoffs was the postseason tourn...


In [21]:
df_playoffs[df_playoffs['title'].str.contains("2023")]

,title,text
74,2023 NBA playoffs,The 2023 NBA playoffs was the postseason tourn...


# Naive OpenAI NBA Playoffs Chatbot
In this session, we are going to built a naive OpenAI chatbot, that captures the information about the year and a question about NBA playoffs and makes a request to OpenAI. 

In [54]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openai.vocareum.com/v1"
)

def chat_with_openai(messages):
    """
    Sends a conversation to OpenAI and gets a response using the updated API.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            
        )
        # Access the content of the response using dot notation
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def naive_OpenAI_chatbot():
    """
    Starts the chatbot interaction.
    """
    print("👋 Welcome to the NBA Playoffs Chatbot!")
    
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": "You are a helpful assistant knowledgeable about NBA playoffs."}
    ]
    
    # Ask the first question
    year = input("Bot: Which NBA playoff year do you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about the NBA playoffs in {year}."})
    
    # Ask the second question
    info_type = input("Bot: What's the information you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about {info_type} in the {year} NBA playoffs."})
    
    # Generate a response
    print("Bot: Let me gather some information for you...\n")
    
    assistant_prompt = f"Generate a detailed paragraph about {info_type} in the {year} NBA playoffs."
    conversation.append({"role": "assistant", "content": assistant_prompt})
    
    response = chat_with_openai(conversation)
    
    if response:
        # Display the generated paragraph
        print(f"Bot: Here's what I found:\n{response}\n")
        
        # Confirm with the user
        is_correct = input("Bot: Is this what you were searching for? (yes/no)\nYou: ").strip().lower()
        
        if is_correct == 'yes':
            print("Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.")
        else:
            print("Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?")
            naive_OpenAI_chatbot()
    else:
        print("Bot: I couldn't retrieve any information. Please try again later.")


## Asking questions to the Naive Chatbot
To test the Naive Chatbot, we are going to ask the following questions:

1. Question 1<br>
    - Year: **2020**
    - Question: **What was the starting five for the two teams in the NBA finals, each player basic statistics?**

2. Question 2
    - Year: **2022**
    - Question: **Which team was the champion and which player was named finals MVP?**

### Question 1 (Naive Chatbot)

In [55]:
naive_OpenAI_chatbot()

👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2020
Bot: What's the information you want to know about?
You:  What was the starting five for the two teams in the NBA finals, each player basic statistics?


Bot: Let me gather some information for you...

Bot: Here's what I found:
The two teams in the 2020 NBA finals were the Los Angeles Lakers and the Miami Heat.

The starting five for the Los Angeles Lakers typically consisted of:
1. LeBron James, who averaged 27.6 points, 10.8 rebounds, and 8.8 assists during the series.
2. Anthony Davis, averaging 25.0 points, 10.7 rebounds and 3.2 assists.
3. Dwight Howard, contributing with 7.5 points, 7.3 rebounds, and 0.7 assists.
4. Kentavious Caldwell-Pope, who averaged 12.8 points, 2.1 rebounds, and 1.3 assists.
5. Danny Green, managing to add 8.0 points, 3.4 rebounds, and 1.3 assists.

The Miami Heat ’s lineup was as follows: 
1. Jimmy Butler, who had a dominant performance with 26.2 points, 8.3 rebounds, and 9.8 assists per game.
2. Bam Adebayo, contributed with 17.8 points, 10.3 rebounds, and 4.4 assists.
3. Jae Crowder, who averaged 12.0 points, 5.6 rebounds, and 1.9 assists.
4. Duncan Robinson, playing well with 12.4 points, 3.3 rebounds, a

Bot: Is this what you were searching for? (yes/no)
You:  yes


Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.


### Question 2 (Naive Chatbot)

In [9]:
naive_OpenAI_chatbot()

👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2022
Bot: What's the information you want to know about?
You:  Which team was the champion and which player was named finals MVP


Bot: Let me gather some information for you...

Bot: Here's what I found:
I'm sorry, but as an AI developed by OpenAI, I can't provide real-time updates or future information. As of the last update in October 2021, the 2022 NBA playoffs hadn't taken place yet, and so the champions and Finals MVP haven't been determined. Please check the most recent and reliable sports news sources for this information.



Bot: Is this what you were searching for? (yes/no)
You:  no


Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?
👋 Welcome to the NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  no
Bot: What's the information you want to know about?
You:  no


Bot: Let me gather some information for you...

Bot: Here's what I found:
I'm sorry, but your question is unclear. Could you please clarify what specific information about NBA playoffs you are interested in? Maybe you're asking about a specific year for the NBA playoffs or a specific team or player? Please give more details so I can assist you better.



Bot: Is this what you were searching for? (yes/no)
You:  yes


Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.


In [ ]:
df_playoffs[df_playoffs['title'].str.contains("2022")]['text'].str.cat(sep='\n')  #

In [56]:
def get_data_from_nba_playoffs(
    query: str, dataset: pd.DataFrame = df_playoffs
) -> str:
    cat_text = dataset[
        dataset["title"].str.contains(str(query))]["text"].str.cat(sep="\n")
    return cat_text

def augmented_openai_chatbot():
    print("👋 Welcome to the Augmented NBA Playoffs Chatbot!")
    
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": "You are a helpful assistant knowledgeable about NBA playoffs."}
    ]
    
    # Ask the first question
    year = input("Bot: Which NBA playoff year do you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about the NBA playoffs in {year}."})
    
    # Ask the second question
    info_type = input("Bot: What's the information you want to know about?\nYou: ")
    conversation.append({"role": "user", "content": f"I want to know about {info_type} in the {year} NBA playoffs."})
    
    # Generate a response
    print("Bot: Let me gather some information for you...\n")

    # Get Data from Wikipedia dataset
    str_filtered_dataset = get_data_from_nba_playoffs(year)
    if str_filtered_dataset != '':
        print(f'''Found information about year "{year}" in NBA playoffs,
        with {len(str_filtered_dataset)} characters.
        ''')
    
    assistant_prompt = f"Generate a detailed paragraph about {info_type} in the {year} NBA playoffs.\n"
    augmented_prompt = assistant_prompt + f"Also use the following wikipedia paragraph as a source of information, if available:\n{str_filtered_dataset[:1024]}"
    conversation.append({"role": "assistant", "content": augmented_prompt})
    print(augmented_prompt)
    response = chat_with_openai(conversation)
    
    if response:
        # Display the generated paragraph
        print(f"Bot: Here's what I found:\n{response}\n")
        
        # Confirm with the user
        is_correct = input("Bot: Is this what you were searching for? (yes/no)\nYou: ").strip().lower()
        
        if is_correct == 'yes':
            print("Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.")
        else:
            print("Bot: I'm sorry it wasn't what you were looking for. Could you clarify or ask something else?")
            naive_OpenAI_chatbot()
    else:
        print("Bot: I couldn't retrieve any information. Please try again later.")


In [ ]:
 - Year: **2022**
    - Question: **Which team was the champion and which player was named finals MVP?**

In [57]:
augmented_openai_chatbot()

👋 Welcome to the Augmented NBA Playoffs Chatbot!


Bot: Which NBA playoff year do you want to know about?
You:  2022
Bot: What's the information you want to know about?
You:  Which team was the champion and which player was named finals MVP?


Bot: Let me gather some information for you...

Found information about year "2022" in NBA playoffs,
        with 55284 characters.
        
Generate a detailed paragraph about Which team was the champion and which player was named finals MVP? in the 2022 NBA playoffs.
Also use the following wikipedia paragraph as a source of information, if available:
The 2022 NBA playoffs was the postseason tournament of the National Basketball Association's 2021–22 season. The playoffs began on April 16 and ended on June 16 with the conclusion of the 2022 NBA Finals. The playoffs also returned to its normal April–June schedule for the first time since 2019, before the COVID-19 pandemic resulted in two postponements in 2020 and 2021.

Overview
Notable updates to postseason appearances
The Phoenix Suns entered the postseason for the second consecutive season and also clinched the best record in the NBA for the first time since 2005.
The Miami Heat entered the postseason for the third consecutive seaso

Bot: Is this what you were searching for? (yes/no)
You:  yes


Bot: I'm glad I could help! Let me know if there's anything else you'd like to know.


In [51]:
str_prompt = '''
Generate a detailed paragraph about Which team was the champion and which player was named finals MVP? in the 2022 NBA playoffs.
Also use the following wikipedia paragraph as a source of information, if available:
The 2022 NBA playoffs was the postseason tournament of the National Basketball Association's 2021–22 season. The playoffs began on April 16 and ended on June 16 with the conclusion of the 2022 NBA Finals. The playoffs also returned to its normal April–June schedule for the first time since 2019, before the COVID-19 pandemic resulted in two postponements in 2020 and 2021.

Overview
Notable updates to postseason appearances
The Phoenix Suns entered the postseason for the second consecutive season and also clinched the best record in the NBA for the first time since 2005.
The Miami Heat entered the postseason for the third consecutive season and also clinched the best record in the Eastern Conference for the first time since 2013.
The Boston Celtics entered the postseason for the eighth consecutive season, the longest such streak active in the NBA at the time.
Boston also won the Eastern Conference and advanced to the NBA Finals for the first time since 2010.
The Utah Jazz and Milwaukee Bucks entered the postseason for the sixth consecutive season.
The Philadelphia 76ers entered the postseason for the fifth consecutive season.
The Brooklyn Nets and Denver Nuggets entered the postseason for the fourth consecutive season.
The Dallas Mavericks entered the postseason for the third consecutive season.
The Memphis Grizzlies and Atlanta Hawks entered the postseason for the second consecutive season.
The Toronto Raptors entered the postseason for the first time since 2020.
The Golden State Warriors entered the postseason for the first time since 2019.
Golden State also won the Western Conference and advanced to the NBA Finals for the first time since 2019 and sixth time since 2015.
The Minnesota Timberwolves and New Orleans Pelicans entered the postseason for the first time since 2018.
The Chicago Bulls entered the postseason for the first time since 2017.
The Los Angeles Lakers missed the postseason for the first time in three years.
The Portland Trail Blazers missed the postseason for the first time in nine years.
The Sacramento Kings missed the postseason for the sixteenth consecutive season, surpassing the Los Angeles Clippers for the longest playoff drought in NBA history, beginning in 2007.

Notable occurrences
Neither Los Angeles team qualified for the playoffs. The last time this occurred was in 2018. (The Los Angeles Clippers made the play-in as the eighth seed but did not advance.)
This is the first time where both No. 8 seeds were eliminated in the Page-McIntyre stage by the No. 9 seeded teams.
A team with a losing record in the Western Conference (Pelicans) qualified for the playoffs. This previously occurred in 2020.
For the first time in NBA playoff history, four players aged 22 or younger (Anthony Edwards, Ja Morant, Tyrese Maxey, and Jordan Poole) scored 30 or more points in a game in the same postseason.
Anthony Edwards joined Derrick Rose, Tyler Herro, and Magic Johnson in scoring at least 35 points in a playoff game before turning 21.
The Utah Jazz became the fifth team in the last 10 postseasons to win a playoff game after trailing by 4-plus points in the final 40 seconds. Teams were 4–701 in that span prior to the game.
The Boston Celtics swept the Brooklyn Nets, marking the 46th year in a row that a sweep occurred. The last time a sweep did not occur was in 1976. This was the 182nd series sweep in NBA playoffs history.
The Philadelphia 76ers and Toronto Raptors series became the first Eastern Conference First Round series to go to a game 6 since 2018.
The Memphis Grizzlies became the first team ever to have multiple comebacks wins after trailing by double-digits going into the fourth quarter in a single playoff series.
The Dallas Mavericks won a playoff series for the first time since their championship run in 2011.
Chris Paul set a playoff record for most consecutive field goals made in a game without a miss, going 14-of-14 from the field.
For the first time since 2019, all top four teams in both the Eastern Conference and the Western Conference made it past the first round of the playoffs.
For the first time since 2011, no first-round matchup went to a game 7.
With a game 3 win at Denver, the Golden State Warriors have won a road game in 24 consecutive playoff series, beating the previous record set by the Miami Heat with 23. This record has since been extended to 27 consecutive playoff series with their road wins against Memphis, Dallas, and Boston.
Game 1 between the Golden State Warriors and Memphis Grizzlies was the first game in playoff history where three players age 22 or younger (Ja Morant, Jaren Jackson Jr., and Jordan Poole) each had 30-plus points.
Ja Morant became the third player in league history to have multiple 45-point games in the postseason before turning 23, joining LeBron James and Kobe Bryant.
The Dallas Mavericks totaled nine assists against the Phoenix Suns in game 5, the lowest number of assists in a playoff game by a team since 2016.
For the first time since 2015, every Conference semifinals series went to a game 6.
The Mavericks–Suns series was the first of the 2022 playoffs to have a game 7, making it the 23rd consecutive NBA postseason to feature a game 7. The last time a game 7 did not occur in the playoffs was 1999. This was the 145th game 7 in NBA playoffs history.
The Golden State Warriors became the first team to grab 70-plus rebounds in a playoff game since 1983.
With the defending champion Milwaukee Bucks losing to the Boston Celtics, the 2022 playoffs was the third straight year where the defending champion was eliminated before the conference finals. After the Celtics defeated the Miami Heat, it was also the fifth straight year with a different EC champion.
With the defending Western Conference champion Phoenix Suns losing to the Dallas Mavericks, this marked the second straight year both of the NBA Finals participants from the previous year were eliminated before the conference finals.
The Dallas Mavericks made their first Conference finals since 2011.
Neither the first-seeded team nor the second-seeded team made the Western Conference finals for the first time since 2011.
In game 4 of the Eastern Conference finals, the Miami Heat failed to get a shot for more than eight minutes, until 3:22 remained in the first quarter. This marked the longest a team has gone with no field goals to start a game in the postseason in 25 years. Subsequently, all starters for the Heat were outscored by a bench player (Victor Oladipo scored 23 points, more than 18 points from all of the starting five). The 18 points was the lowest number combined in a playoff game since starters were first tracked in 1971 and was also the first time in which a bench player outscores the entire starting lineup.
By advancing to the NBA Finals, the Golden State Warriors made their sixth Finals appearance in eight seasons, the first since 1990–98 Chicago Bulls (led by Michael Jordan) to have this feat.
Stephen Curry joined LeBron James and Michael Jordan as the only players in NBA Finals to score 40 or more points at age 34 or older.
Jayson Tatum set the all-time NBA record with 100 turnovers in a single postseason.
By beating the Celtics, the Warriors became the only team with a non-losing series record (1–1) against the Celtics in the Finals. The Celtics had winning series records in the Finals against the Lakers (9–3), the Hawks (3–1), the Rockets (2–0), the Bucks (1–0) and the Suns (1–0).

Format
Eight teams from each conference participated in the playoffs. The top six teams in each conference, based on winning percentage, directly qualified for the playoffs; the seeding order of those teams was also based on winning percentage. If two or more teams had the same record, standard NBA tiebreaker rules were used.
The NBA Board of Governors approved a format for the 2021–22 season to have a play-in tournament involving the teams ranked 7th through 10th in each conference. The 7th-place team and 8th-place team participated in a "double-chance" game, with the winner advancing to the playoffs as the 7-seed. The loser then played the winner of the elimination game between the 9th-place and 10th-place teams to determine the playoff's 8-seed. The NBA's regular playoff format then proceeded as normal.
Each conference's bracket was fixed with no reseeding. All rounds were best-of-seven series; a series ended when one team won four games, and that team advanced to the next round. All rounds, including the NBA Finals, were in a 2–2–1–1–1 format with regards to hosting. In the conference playoffs, home court advantage went to the higher-seeded team (number one being the highest). Conference seedings were ignored for the NBA Finals: home court advantage went to the team with the better regular season record, and, if needed, ties were broken based on head-to-head record, followed by intra-conference record.

Playoff qualifying
On March 9, 2022, the Phoenix Suns became the first team to clinch a playoff spot. On April 5, the San Antonio Spurs and New Orleans Pelicans became the last two teams to qualify for postseason play, as both teams secured spots in the first-stage play-in tournament. On April 7, the Denver Nuggets became the final team to secure a direct berth in the playoff bracket, qualifying as the sixth seed in the Western Conference and relegating the Minnesota Timberwolves to the play-in tournament. While noted in the below tables, division titles have no bearing on seeding.
Seeds 7 and 8 in each conference were determined via the first-stage play-in tournament, held April 12–15.

Eastern Conference
Cleveland (44–38) and Charlotte (43–39) also secured play-in berths but did not advance to the playoffs.

Western Conference
The L.A. Clippers (42–40) and San Antonio (34–48) also secured play-in berths but did not advance to the playoffs.

Play-in tournament brackets
Eastern Conference
Bold Game winner
Italic Team with home-court advantage

Western Conference
Bold Game winner
Italic Team with home-court advantage

Bracket
Teams in bold advanced to the next round. The numbers to the left of each team indicate the team's seeding in its conference, and the numbers to the right indicate the number of games the team won in that round. The division champions are marked by an asterisk. Teams with home court advantage, the higher-seeded team, are shown in italics.

First round
Note: Times are EDT (UTC−4) as listed by the NBA. If the venue is located in a different time zone, the local time is also given.

Eastern Conference first round
(1) Miami Heat vs. (8) Atlanta Hawks
Duncan Robinson led the Heat in scoring with 27 points on 9-of-10 shooting, including going 8-of-9 from three-point range. On the other hand, Atlanta's Trae Young struggled, shooting just 1-of-12, finishing the game with eight points and six turnovers.
 Jimmy Butler scored a playoff career-high 45 points, shooting 15-of-25 as well as committing zero turnovers en route to securing a 2–0 series lead. Trae Young also rebounded in game 2, finishing the game with 25 points, but he committed a career-high 10 turnovers.
 In danger of going down 0–3 in the series, Trae Young hit a game-winning floater in the lane with 4.4 seconds left as the Hawks overcame a 16-point deficit in the second half.
 After losing Kyle Lowry in game 3 with a hamstring strain, the Heat dominated the Hawks down the stretch, outscoring Atlanta in the final three quarters 85–60. Jimmy Butler scored 13 of his 36 points in the second quarter along with 10 rebounds and four assists. Trae Young scored only nine points with five assists and five turnovers, as he was held to single digit points by Miami's defense for the second time in four games.
 Without Jimmy Butler and Kyle Lowry, the Heat were able to hold off the Hawks thanks to a 23-point performance from Victor Oladipo. De'Andre Hunter shined for Atlanta in the loss, scoring a career-high 35 points, as well as the most points scored by a Hawks player in the series.

This was the third playoff meeting between these two teams, with the Hawks winning the first two meetings.

(2) Boston Celtics vs. (7) Brooklyn Nets
The Celtics lost a 15-point third quarter lead but went on to win game 1 on Jayson Tatum's layup at the buzzer.
 The Nets had a 17-point lead late in the first half, but lost the game due to a huge fourth quarter by the Celtics. Kevin Durant notably struggled in the second half, going 0–10 from the floor and committing 4 turnovers as Boston outscored Brooklyn 59–42 in the final two quarters.
 Despite a stellar night from Bruce Brown, who had 26 points on 10-of-19 shooting from the field, the Nets were unable to overcome poor performances from Durant and Irving, who only scored 16 points apiece, as well as a 39-point explosion from Jayson Tatum who also had 6 steals. The Celtics led by as many as 15 in the final quarter before holding off the Nets to take a commanding 3–0 series lead.
 Despite Jayson Tatum fouling out late in the fourth quarter, the Celtics were able to hold off the Nets to complete the sweep. Kevin Durant had his best game of the series with 39 points, helping Brooklyn trim the deficit to 1 point with 3:29 to play in the fourth quarter, but thanks to Tatum's 29 points, Boston never trailed. Nic Claxton struggled at the free throw line, missing his first 10 attempts, and finishing 1-of-11. This was the eighth playoff sweep in Celtics' franchise history, as Boston swept its first-round series for the third time in four years. The Nets were the only team in this season's playoffs to get swept.

This was the fourth playoff meeting between these two teams, and the second since the New Jersey Nets relocated to Brooklyn in 2012, with the Nets winning the first three meetings.

(3) Milwaukee Bucks vs. (6) Chicago Bulls
Despite finishing the game with 21 turnovers and blowing a 16-point lead, the Bucks outlasted the Bulls 93–86. Giannis Antetokounmpo finished the game with 27 points and 16 rebounds. The Bulls struggled on offense, shooting a season-low 32.3% from the floor, with DeMar DeRozan and Zach LaVine shooting a combined 12-of-44 (27.3%).
 While the Bulls were leading by as much as 18 in the third quarter, they withstood two furious rallies in both the third and fourth quarters by the Bucks, with DeMar DeRozan scoring a playoff career-high 41 points. This was the Bulls' first playoff victory since game 2 of the 2017 Eastern Conference first round against the Boston Celtics. Two Bucks players suffered injuries during the game: Bobby Portis left mid-game after suffering a "right eye abrasion" and Khris Middleton suffered a sprained MCL in his left knee.
 Despite Chicago hosting their first playoff game in five years, Milwaukee dominated throughout the game. Grayson Allen came off the bench and led the Bucks in points with 22, along with Bobby Portis collecting 16 rebounds, tying a career high. The loss was also the largest playoff loss at home in Bulls history.
 Grayson Allen set playoff career highs with 27 points and six three-pointers, along with Giannis Antetokounmpo leading the team with 32 points and 17 rebounds as the Bucks cruised to a second straight win on the road. With the loss, the Bulls had been outscored by a combined 54 points over the past two games, both of which were at home.
 Despite being without Khris Middleton the last three games, Milwaukee won all three by an average margin of 23.3 points. DeMar DeRozan was scoreless in the first 26 minutes of the game and attempted just one shot in the first quarter, where Chicago was outscored 34–18. Giannis Antetokounmpo had 33 points and nine rebounds as the Bucks won their first-round matchup for the fourth consecutive season.

This was the fifth playoff meeting between these two teams, with each team winning two out of the first four meetings.

(4) Philadelphia 76ers vs. (5) Toronto Raptors
In his first playoff game as a starter, Tyrese Maxey scored 38 points, with 21 of his 38 points coming in the third quarter. Raptors rookie Scottie Barnes had 15 points, 10 rebounds, and eight assists prior to exiting the game with an ankle injury.
 After the Raptors took a one-point lead into the second quarter, the 76ers dominated for the rest of the game. Philadelphia outscored Toronto 80–64 in the final three quarters, with Joel Embiid finishing the game with 31 points and 11 rebounds.
 With 0.9 seconds on the shot clock and 2.6 seconds remaining in overtime, Joel Embiid hit a game-winning three-point shot to secure a commanding 3–0 series lead after overcoming a 17-point deficit and never leading in regulation.
 Scottie Barnes made his return to the lineup for game 4 after missing the previous two games, leading the team with 11 rebounds. Raptors' Pascal Siakam scored 34 points while the 76ers' James Harden scored 22 points. In the second quarter, Fred VanVleet exited the game due to a hip injury and did not return. Despite this, the Raptors staved off elimination with a 110–102 win to send the series back to Philadelphia.
 Despite being without Fred VanVleet, the Raptors, with the help of Pascal Siakam's 23 points, 10 rebounds, and seven assists led almost the entire game and staved off elimination again to send the series back to Toronto for game 6. The 76ers had their own share of issues: while Joel Embiid scored 20 points with 11 rebounds, and James Harden had seven assists; the 76ers' bench scored only five points.
 The 76ers completed a 35-point rout of the Raptors to eliminate them from the playoffs. Joel Embiid had 33 points and 10 rebounds, and teammate Tyrese Maxey scored 25. The Raptors' Pascal Siakam scored 24 points in the loss, while Chris Boucher came off the bench and had 25 points and 10 rebounds.

This was the third playoff meeting between these two teams, with each team winning one series.

Western Conference first round
(1) Phoenix Suns vs. (8) New Orleans Pelicans
The Pelicans started off slowly, but chipped away at a 23-point deficit to eventually trail 79–71 by the end of the third. However, Chris Paul took over late in the game by scoring 19 of his 30 points in the fourth quarter.
 After scoring 31 points in the first half, Devin Booker left the game in the third quarter after suffering a "mild hamstring strain" and did not return to the game. Meanwhile, Brandon Ingram finished the game with 37 points, 11 rebounds, and nine assists, leading the Pelicans to their first playoff win since 2018.
 Without Devin Booker, the Suns outlasted the Pelicans on the road, with Chris Paul and Deandre Ayton combining for 56 points. Paul scored 15 of Phoenix's first 23 points of the final quarter, along with committing zero turnovers, his second straight game doing so.
 Brandon Ingram scored 16 of his 30 points in the third quarter, his third straight game with 30-plus points. Jonas Valančiūnas also contributed with 26 points and 15 rebounds, along with scoring ten points in a 3:34 span in the fourth quarter to help the Pelicans pull away. Chris Paul struggled for the Suns, scoring just four points and three turnovers after previously having just two turnovers in the first three games.
 With the series back in Phoenix, Mikal Bridges helped the Suns take a 3–2 series lead thanks to a playoff career-high 31 points and four blocks, all while playing 47 minutes in the game. Chris Paul rebounded in game 5 as well, tallying 22 points and 11 assists. Phoenix never trailed in the game.
 In Devin Booker's return from the hamstring injury that had kept him out the previous three games, Chris Paul scored 33 points on a perfect 14-for-14 from the field (including one three-pointer) and 4-for-4 from the free throw line as the Suns advanced. Paul's 14-for-14 shooting set a record for most field goals made in a playoff game without a miss, and the first player to make 13-plus shots in a playoff game without a miss.

This was the first playoff meeting between the two teams.

(2) Memphis Grizzlies vs. (7) Minnesota Timberwolves
In his playoff debut, Anthony Edwards led the Timberwolves in scoring with 36, joining Derrick Rose, Tyler Herro, and Magic Johnson in scoring at least 35 points in a playoff game before turning 21. The win was also the Timberwolves' first playoff opener they have won since 2004.
 Ja Morant and the Grizzlies rebounded in game 2, with Morant being one rebound shy of his first career playoff triple-double. The win was also the widest margin of victory in the Grizzlies' playoff history.
 Despite trailing by 25-plus points on two occasions, the Grizzlies battled back late in the game, including going on a 42–12 run in the final 13 minutes of the game to secure their first lead of the series. Ja Morant also became the first player in the Grizzlies' playoff history to record a triple-double.
 Karl-Anthony Towns and the Timberwolves were able to rebound after their game 3 collapse, with Towns putting up a career playoff-best 33 points and 14 rebounds. Desmond Bane also had a career night, hitting eight three-pointers and leading the Grizzlies with 34 points, tying a career high.
 After the Grizzlies erased a 13-point fourth-quarter lead by the Timberwolves, the game would come down to the wire from that point on. At the eight-second mark, Ja Morant would make two clutch free throws. After Minnesota used their last timeout, Anthony Edwards would hit a clutch three-pointer to tie the game at 109 with 3.7 seconds left. After Memphis called a timeout, Morant hit a game-winning layup with one second left. This was the Grizzlies' second double-digit comeback victory of the series.
 The Grizzlies' rallied from a double-digit deficit entering the fourth quarter for the third time in the series, winning their opening playoff series for the first time in seven years. Desmond Bane and Dillon Brooks each scored 23 points, with Bane going 27-of-56 on three-pointers in the series.

This was the first playoff meeting between the two teams.

(3) Golden State Warriors vs. (6) Denver Nuggets
In his playoff debut, Jordan Poole led the Warriors in points with 30, shooting 9-of-13. Stephen Curry also came off the bench to contribute 16 points in 21 minutes after missing exactly a month with a sprained ligament in his left foot. This was also the Warriors' first playoff win since game 5 of the 2019 NBA Finals. Also, the city of San Francisco hosted a playoff game for the first time since 1964.
 Stephen Curry came off the bench for a second straight game, dropping 34 points in 23 minutes to lead the Warriors to a 2–0 advantage in the series. Nikola Jokić also led the Nuggets with 26 points and 11 rebounds before being ejected with seven minutes left in the game.
 The Warriors finished the game on a 9–2 run to hand the Nuggets their seventh consecutive loss in the playoffs. Stephen Curry and Jordan Poole combined for 54 points, along with Klay Thompson shooting six three-pointers en route to a 3–0 series lead.
 The Nuggets finished the game on a 9–5 run, snapping their seven-game losing streak in the playoffs, dating back to the 2021 postseason. Nikola Jokić led all scorers with 37 points and assisted on a Will Barton three-pointer from the left corner with 8.3 seconds left, effectively putting the game away.
 The Warriors advanced to the Western Conference semifinals for the first time since the 2019 NBA playoffs. Stephen Curry scored 20 points in the second half, leading the Warriors with 30 points. Gary Payton II also scored 10 of his 15 points in the fourth quarter. Nikola Jokić scored 12 of his 30 points in the final 3:46 of the fourth quarter, finishing the game with 19 rebounds and eight assists as the Nuggets lost their eighth game in their last nine playoff games.

This was the second playoff meeting between the two teams, with the Warriors winning the first meeting.

(4) Dallas Mavericks vs. (5) Utah Jazz
Scoring just two points in the first half, Donovan Mitchell erupted in the second half with 30 points, leading the Jazz to a game 1 win on the road. This was also the first time the Mavericks started the playoffs at home since their championship run in 2011.
 Down 0–1 in the series and without their star Luka Dončić, the Mavericks won due to a career-high 41 points from Jalen Brunson, along with the franchise record of fewest turnovers ever in a postseason game, with 3.
 Despite Donovan Mitchell putting up 28 points in the second half, the Mavericks took their first lead of the series, with the Dallas bench outscoring the Utah bench 49–24. The Jazz had previously won 11 straight against the Mavericks at home coming into the game.
 Luka Dončić made his return after missing the first three games of the series due to a "calf strain" and scored a double-double with 30 points, 10 rebounds, and 4 assists. Despite this, however, the game came down to the wire. With 30 seconds left in the game with the Mavericks up one, Dwight Powell was fouled but missed 2 crucial free throws, then Donovan Mitchell lobbed the ball to Rudy Gobert for an alley-oop, giving the Jazz the lead with 11 seconds to go. Dallas had one more shot, but Doncic's  potential game-winning three-pointer fell short as Utah tied the series 2-all. The Jazz also became the fifth team since 2013 to win after trailing by four-plus points in the final 40 seconds. Teams were 4–701 in that span coming into the game.
 After the first four games were decided by eight points or less, the Mavericks routed the Jazz at home 102–77 to regain the series lead. Luka Dončić once again led the Mavericks in points (33), rebounds (13), and assists (5) for the second straight game. Donovan Mitchell, who averaged 30 points in the first four games, scored just nine and missed all seven of his three-pointers before leaving with a left hamstring injury in the fourth quarter.
 Though the Jazz had a 12-point lead at halftime, the Mavericks stormed back to take the lead in the final few minutes. Rudy Gobert made a layup with 35 seconds to go to reduce the deficit to just one point. However, after a Mike Conley turnover, Jalen Brunson doubled the Mavericks' lead by scoring a free throw at 4.3 seconds. Bojan Bogdanović then missed a game-winning three-pointer with 1.9 seconds left on the clock to hand the Mavericks their first series win since their championship run in 2011. Donovan Mitchell recovered from the injury he sustained in the previous game, and had 23 points, eight rebounds, and nine assists. Brunson and Luka Dončić each scored 24 points for Dallas, and Spencer Dinwiddie scored 19 points.

This was the third playoff meeting between these two teams, with the Mavericks winning the first two meetings.

Conference semifinals
Note: Times are EDT (UTC−4) as listed by the NBA. If the venue is located in a different time zone, the local time is also given.

Eastern Conference semifinals
(1) Miami Heat vs. (4) Philadelphia 76ers
Without Joel Embiid, who suffered an orbital fracture and concussion in the final minutes of game 6 against Toronto, the 76ers were unable to overcome missing their star player, with Philadelphia shooting 17.6% from beyond the arc, their second-worst percentage of the season. Tyler Herro scored 25 points—the second most he has ever had in a playoff game—while Bam Adebayo finished with 24 points and 12 rebounds as the top-seeded Heat outscored the 76ers 56–41 in the second half.
 The 76ers were unable to rally without their star Joel Embiid for the second straight game, with four Heat players scoring 18-plus points (Adebayo, Butler, Herro and Oladipo), along with shooting 48.3% from the three-point line. Tyrese Maxey led the way for Philadelphia with 34 points, but a 10–0 run in the fourth turned an eight-point Miami lead into an 18-point advantage, sealing the win.
 Although listed as "doubtful" leading into the game, Joel Embiid made his first appearance of the series, leading the 76ers to their first win with 18 points and 11 rebounds. Danny Green rebounded from a poor shooting performance in game 2 (shooting 1-for-10), making 6-of-7 three-pointers through the first three quarters. Kyle Lowry also returned from injury, but finished the game scoreless, with Jimmy Butler being the only Heat player to score more than 14 points in the game.
 After being held for under 30 points in 13 consecutive playoff games, James Harden broke through with 31 points to help Philadelphia tie the series. Joel Embiid once again contributed with 24 points and 11 rebounds as the 76ers shot 48.5% from the three-point line. Jimmy Butler also scored 40 points on 13-of-20 shooting from the floor, but the Heat, who shot 38% on three-pointers during the season, shot just 7-of-35 (20%) from beyond the arc.
 The Heat outscored the 76ers by 35 points, tying the second-widest margin of victory in Heat playoff history and tying the widest margin of victory of the 2022 NBA playoffs at the time of the game. Seven players for the Heat scored in double-digits, with Jimmy Butler scoring 23 points and Max Strus with 19 points and 10 rebounds. None of Philadelphia's starters played in the final 8:18 of the game, as the Heat outscored the 76ers 39–19 in the fourth quarter.
 Jimmy Butler scored 32 points and led the Heat to the conference finals for the second time in three seasons, with Butler scoring 14 points on 6-of-9 shooting in the third quarter where the Heat used a 16–2 run to take control. Joel Embiid scored 20 points with 12 rebounds on 7-of-24 shooting, but teammate James Harden failed to score in the second half, scoring all 11 of his points in the first half. The loss marked the fourth time in the last five years the 76ers had lost in the conference semifinals, having last reached the conference finals in 2001.

This was the third playoff meeting between these two teams, with each team winning one previous series.

(2) Boston Celtics vs. (3) Milwaukee Bucks
The Bucks won their fourth consecutive playoff game thanks to Giannis Antetokounmpo's second career playoff triple-double, along with Jrue Holiday leading the team with 25 points and 10 rebounds. Jayson Tatum scored 21 and Jaylen Brown had 12 points, but combined to shoot 10-for-31 in the game.
 Despite being without Marcus Smart, the Defensive Player of the Year, the Celtics held the Bucks to just 86 points, with Milwaukee shooting 16.7% from the three-point line. Jaylen Brown rebounded in game 2, scoring 25 of his 30 points in the first half, including 5-for-5 from the three-point line.
 Boston trailed by as much as 14 late in the third quarter and was facing a 13-point deficit with less than 10 minutes left before storming back. The Celtics pulled ahead 100–99 on Jaylen Brown's two free throws with 1:49 remaining. However, after Giannis Antetokounmpo made a go-ahead basket with 44.3 seconds left, Marcus Smart was fouled just before attempting a potential tying three-pointer with 4.6 seconds left. After making the first free throw, Smart missed the second intentionally and got his own rebound, but missed his putback attempt. After a brief scramble, Al Horford got the rebound and banked it in, but his final shot came just after the buzzer sounded, giving Milwaukee the win.
 The Celtics tied the series thanks to Al Horford and Jayson Tatum scoring 30 points apiece. Horford had 16 points and Tatum 12 in the fourth quarter alone to help the Celtics outscore the defending NBA champions at home 43–28. Giannis Antetokounmpo led the way for the Bucks with 34 points and 18 rebounds, but Jrue Holiday notably struggled from the field, shooting 5-for-22 (22.7%) as the Bucks shot 41.5% from the field as a team.
 With Boston looking to go up 3–2 in the series and leading by as much as 14 by the start of the fourth quarter, Milwaukee mounted a furious comeback in the final minutes. Down by 6 with 1:45 left in the game, both Giannis Antetokounmpo and Jrue Holiday each hit three-pointers to tie the game at 105. After Jayson Tatum was fouled and made both free throws with 31 seconds left, Antetokounmpo would also be fouled, making the first but missing the second. However, Bobby Portis secured the rebound and scored with 11.4 left, while Marcus Smart's potentially go-ahead shot was blocked by Holiday, who then converted two free throws to make it a three-point game with 5.9 left. Smart would be stripped of the ball by Holiday again at midcourt, completing the Bucks' comeback victory.
 Looking to move on from their loss in game 5 and looking to avoid elimination on the road, the Celtics were able to control the game, leading by as much as 18. Once again, however, the Bucks tried to mount another rally, cutting the deficit to four, but the Celtics were able to respond and force a game 7 in Boston. Jayson Tatum led the way with 46 points, nine rebounds and four assists, while Al Horford recorded 10 rebounds and four blocks for the Celtics. Giannis Antetokounmpo also had a big performance in the Bucks' loss, scoring 44 points, six assists, and 20 rebounds—the first 40–20 playoff game since Shaquille O'Neal achieved it in game 1 of the 2001 NBA Finals.
 The Bucks controlled most of the first quarter, leading by as much as 10, but the Celtics would storm back to lead by five points at halftime. Then, the Celtics took over the game from that point on as they beat the defending champions by 28 points. Grant Williams scored a career-high 27 points, including seven three-pointers while Jayson Tatum scored 23 points, Al Horford grabbed 10 rebounds, Celtics bench player Payton Pritchard scored 14 points, including four three-pointers, and Marcus Smart had 10 assists. As a team, the Celtics connected on a game 7 record 22 made three-pointers.
Despite the Bucks' loss, Giannis Antetokounmpo secured a double-double with 25 points, 20 rebounds, and 9 assists, barely missing out on a triple-double. The Bucks missed 24 of their last 25 attempts from three-point range and finished the game 4-of-33 from beyond the arc in game 7, which is the second-worst three-point percentage in a playoff game ever with 30-plus attempts. Antetokounmpo would become the first player with 200 points, 100 rebounds, and 50 assists in a playoff series.

This was the eighth playoff meeting between these two teams, with the Celtics winning five of the first seven meetings.

Western Conference semifinals
(1) Phoenix Suns vs. (4) Dallas Mavericks
After jumping to a 9–0 lead in the opening minutes, the Suns never trailed in the game as they beat the Mavericks for the 10th straight time, dating back to 2019. All five Suns starters scored in double figures, with Deandre Ayton leading the way with 25 points. Luka Dončić led the Mavericks with 45 points, 12 rebounds and eight assists, but no other Mavericks starter scored more than 15 points.
 Leading 89–83 entering the fourth quarter, the Suns exploded on offense, shooting 84% from the field (16-of-19) and outscoring Dallas 40–26. Chris Paul and Devin Booker combined for 58 points, with Paul scoring 14 of his 28 in the fourth quarter, as the Suns shot a franchise playoff-record 64.5% from the field. Luka Dončić scored 35 points, but only one other starter for the Mavericks scored more than 9 points.
 After struggling in the first two games of the series, Jalen Brunson scored 28 points and helped the Mavericks snap an 11-game losing streak against the Suns. Luka Dončić also had 26 points to go with 13 rebounds and nine assists, one assist shy of his fourth-career playoff triple-double. For the Suns, Chris Paul struggled on his 37th birthday, committing seven turnovers in the first half, the most turnovers Paul's ever had in any half in his career.
 The Mavericks tied the series thanks to a playoff career high from Dorian Finney-Smith, who scored 24 points on 8-of-12 from three-point range. Dallas matched its 20 three-pointers (20-of-44) from a similar Mother's Day rout of the Los Angeles Lakers in 2011, a win that completed a four-game sweep in the second round. Devin Booker scored 35 points for the Suns, but Chris Paul was in foul trouble the majority of the game, eventually fouling out early in the fourth quarter, finishing the game with 5 points and 6 fouls.
 After leading by 3 coming out of halftime, the Suns outscored the Mavericks 61–34, including a 17–0 run in the third quarter as the Suns won their fourth consecutive playoff game at home. Devin Booker led the Suns in points with 28, bringing his playoff career total to 804 points in 30 career playoff games. Luka Dončić finished with 28 points and 11 rebounds and Jalen Brunson added 21 points. However, the Mavericks shot just 38% from the field as a team, making just 9 assists, the lowest number of assists by a team in the playoffs since 2016.
During the final seconds of the game, Suns center Bismack Biyombo was fouled on a dunk attempt by Mavericks forward Marquese Chriss. The two had a verbal altercation after the foul, and both players were subsequently ejected. After he was ejected, Chriss ran into the same tunnel that Biyombo had gone into, and the two exchanged further words. Security guards and Mavericks center Boban Marjanović helped separate the two players from each other.

 After previously going 0–2 in elimination games in his career, Luka Dončić scored 33 points and 11 rebounds to help the Mavericks force a game 7, with Dallas going on a decisive 19–5 run to finish the first half. Devin Booker scored 19 points, but was shut out from 3-point range (0-of-4) and committed a season-high eight turnovers, with the Suns shooting a playoff-low 40% with 22 turnovers, also a playoff-worst. Chris Paul also struggled, having more turnovers (5) than assists (4) for the second time in the last four games.
 This was the Suns' first game 7 since the 2006 Western Conference semifinals. In a series where every home team won their game, game 7 had a different outcome. The Mavericks dominated the entire game, never giving the Suns the lead, eventually going into halftime leading 57–27; the halftime deficit of 30 being the largest in game 7 history. The lead would later grow to as much as 46 as the Mavericks upset the Suns in 7 games, and advanced to the Western Conference finals for the first time since their championship run in 2011.
Luka Dončić shined in the victory with a double-double (35 points, 10 rebounds, and 4 assists), while Jalen Brunson and Spencer Dinwiddie scored 24 and 30 points, respectively. Devin Booker had 11 points, and Chris Paul had 10 points and 4 assists as the duo were held to a combined 7-for-22 from the field. Chris Paul also became the first player ever to be on five teams that blew 2–0 leads in a best-of-7 series, along with losing his 4th consecutive game 7.

This was the third playoff meeting between these two teams, with each team winning one series.

(2) Memphis Grizzlies vs. (3) Golden State Warriors
Klay Thompson hit a go-ahead 3-pointer with 36 seconds left, and the Warriors overcame Draymond Green's first-half ejection to edge the Grizzlies 117–116. Jordan Poole led the Warriors with a career-high 31 points off the bench, hitting a playoff-best 5-of-10 3-pointers. Jaren Jackson Jr. matched his season high with six 3-pointers and had a playoff career-high 33 points. Ja Morant also led the team with 34 points and 10 assists, but missed the potential go-ahead layup as time expired.
 Ja Morant scored 47 points to match his postseason high and tie the series with a 106–101 victory over the Warriors. Morant scored the last 15 points for Memphis, starting with 4:16 left, along with hitting a 7-foot jumper with 1:42 left to permanently give Memphis the lead. Morant also became the third player in league history to have multiple 45-point games in the postseason before turning 23, joining LeBron James and Kobe Bryant.
 The Warriors scored over 140 points in a playoff game for the first time since 1967 as they routed the Grizzlies 142–112. Stephen Curry scored 30 points, Klay Thompson had 21 and nine rebounds and Golden State shut down the Memphis supporting cast, with no other Grizzlies player besides Ja Morant scoring more than 16 points. Morant had 34 points prior to injuring his right knee midway through the fourth quarter.
 With Warriors head coach Steve Kerr out after testing positive for COVID-19 and Ja Morant out for Memphis after injuring his right knee, Mike Brown took over head coaching duties for the Warriors and led Golden State on an 11–5 run to end the game, with Stephen Curry converting eight free throws over the final 45.7 seconds to finish the game with 32 points. The Warriors scored 39 points in the fourth, topping their 38 the entire first half. Jaren Jackson Jr. and Dillon Brooks combined for 33 points for the Grizzlies, but shot a combined 12-of-40 (30%) from the field.
 Facing elimination at home and their star Ja Morant doubtful for the remainder of the playoffs, the Grizzlies dominated game 5, leading by as much as 55 points at one point. Jaren Jackson Jr., Desmond Bane and Tyus Jones each scored 21 points as Memphis set a new record for widest margin of victory in a game in Grizzlies playoff history, as well as the widest margin of victory in the 2022 playoffs. The Warriors also lost for the first time with assistant Mike Brown filling in for coach Steve Kerr, who missed his second straight game since testing positive for COVID prior to game 4. Klay Thompson and Stephen Curry combined for 33 points, but Golden State turned the ball over 22 times, the most turnovers they had had in a game since 2014.
 Klay Thompson knocked down eight 3-pointers en route to 30 points and Stephen Curry scored 29 with six 3-pointers to send the Warriors to the Western Conference finals for the sixth time in the last eight years. Draymond Green also contributed 14 points, 15 rebounds and eight assists, along with Kevon Looney returning to the starting lineup and grabbing 22 rebounds, a career high. The Warriors also out-rebounded the Grizzlies 70–44, becoming the first team to grab 70-plus rebounds in a playoff game since 1983 and have 25 offensive rebounds, the most since 1987.

This was the second playoff meeting between the two teams, with the Warriors winning the first meeting.

Conference finals
Note: Times are EDT (UTC−4) as listed by the NBA. If the venue is located in a different time zone, the local time is also given.

Eastern Conference finals
(1) Miami Heat vs. (2) Boston Celtics
Jimmy Butler scored 27 of his 41 points in the second half, and a huge third quarter by the Heat—in which Butler outscored the Celtics by himself—carried them to an 118–107 win. Jayson Tatum had 29 points and Jaylen Brown had 24, but were without starters Marcus Smart (foot) and Al Horford (health and safety protocols). Along with being outscored 39–14 in the third quarter, Boston failed to score a field goal in the first six minutes of the second half—the slowest start to a half in any of the Celtics' 94 games this season.
 Jayson Tatum scored 27 points, Marcus Smart and Jaylen Brown each had 24 as the Celtics went on a massive first-half run to roll past the Heat on the road. Boston used a 17–0 run late in the first quarter—fueled by five 3-pointers in the span of six possessions—to take control. Smart was also a rebound shy of a triple-double, after adding 12 assists and nine rebounds. Jimmy Butler had 29 points in 32 minutes, but no other player for Miami scored more than 14 points as the Heat suffered their first loss at home in these playoffs.
 After a blowout loss in Miami that cost them home-court advantage in the Eastern Conference finals, the Heat jumped to a 26-point first-half lead over the Celtics in game 3. Bam Adebayo scored 31 points with 10 rebounds, filling a void left by injured all-star Jimmy Butler, while also putting up six assists and four of the Heat's franchise postseason record 19 steals. The Celtics never led, but they cut a 46–20 deficit to 1 point on a 3-pointer from Jaylen Brown, who finished with 40 points.
 Despite being without Marcus Smart due to a right ankle sprain, the Celtics jumped out to a 26–4 first quarter lead and never trailed en route to dominating the Heat in game 4. Jayson Tatum led all scorers with 31 points, and Al Horford was masterful defensively for the Celtics, grabbing 13 rebounds and blocking four shots.
For Miami, they scored just 52 points in the first three quarters, with the Heat not successfully making a shot from the field until 3:21 left in the first quarter, the longest drought without a field goal to start a playoff game in the last 25 years. Victor Oladipo also came off the bench and outscored the entire Miami starting lineup, with the 18 points from the Heat starting five being the fewest points scored by a team's starters in a playoff game since starters were first tracked in 1971, and the first time an entire starting lineup was outscored by a player from its own bench.

 With Marcus Smart returning to the starting lineup for Boston and Tyler Herro ruled out for Miami due to a groin injury, the first half was a low-scoring affair with the score at halftime 42–37 in favor of the Heat. The game remained close until the 3rd quarter with 2:44 left. As the Celtics were ahead 59–58, they increased their lead with a 24–2 run which continued into the 4th quarter, eventually putting the game away as the Celtics became one game away from advancing to their first NBA Finals in 12 years. Jaylen Brown led the Celtics with 25 points while Jayson Tatum secured a double-double with 22 points, 12 rebounds and nine assists. Tatum (24 years and 83 days old) became the second-youngest player in NBA history to reach 1,500 playoff points.
Meanwhile, the Heat shot 7-of-45 (15.6%) from three in game 5, which was the second-lowest three-point field goal percentage in NBA playoff history with 40-plus attempts. The Miami backcourt also struggled as Kyle Lowry and Max Strus went a combined 0-of-15 in the game, which was the worst "zero-fer" for a starting backcourt in a playoff game since starters were officially tracked in 1971.

 Coming into the game, many fans and experts believed that the season for the Heat was going to end after their lackluster performances in games 4 and 5, with Golden State Warriors player Draymond Green stating "We're going to play Boston" in response to who he would rather face in the 2022 NBA Finals.
With Miami facing elimination, Jimmy Butler scored a playoff career-high 47 points (shooting 16-for-29) to go with 9 rebounds, 8 assists, and four steals as the Heat forced a game 7. Kyle Lowry also had his best game of the series, scoring 18 points and 10 assists before fouling out late in the fourth quarter. For Boston, Jayson Tatum had 30 points and nine rebounds, while Derrick White came off the bench to score 11 of his 22 points in the fourth quarter. Jaylen Brown also scored 20 points for Boston, but missed a crucial pair of free throws with the game tied at 99 as the Heat finished the game on a 17–6 run.
Many fans and media members compared Butler's performance to that of LeBron James when he was with the Heat during game 6 of the 2012 Eastern Conference finals (also against the Celtics), in which he scored 45 points, 15 rebounds & five assists on the road while facing elimination.

 Boston opened up on a 12–3 run and had a 15-point lead after one quarter. The Celtics led by 17 early in the second quarter, but the Heat would pull within 6 by halftime. The Heat cut the lead to 2 early in the third, but the Celtics responded to every second-half Miami rally, leading by as many as 14. Although the Heat cut the lead to just 3 points with 10 minutes left, the Celtics rebuilt their lead, and were up 98–85 with 3:25 left. The Heat then went on a furious 11–0 run in 3 minutes, bringing the score to 98–96. After Marcus Smart missed a driving layup with 20 seconds left, the Heat had a chance to tie or take the lead. However, Jimmy Butler missed a three-point shot, and Al Horford grabbed the defensive rebound, leading to two free throws that made it a 4-point game with 11 seconds to go. The Heat failed to score, and the Celtics won 100–96, with Boston never trailing in the game.
With the win, the Celtics advanced to the NBA Finals for the first time since 2010 and for the 22nd time in their franchise history. Horford, in his 15th season, shed a record for the most playoff games without an NBA Finals appearance, at 141.
Jayson Tatum won the inaugural Eastern Conference finals MVP award, averaging 25.0 points, 8.3 rebounds, and 5.6 assists on 46% from the field, 35% from beyond the arc, and 86% from the free throw line.
Notably, Mark Jones filled-in for Mike Breen as play-by-play commentator in game 7, with Jeff Van Gundy and Mark Jackson intact as analysts; Breen was out due to a positive COVID-19 test. Originally, Jones was to call the game on ESPN Radio with Doris Burke. However, with changes in the commentary team, Marc Kestecher joined Burke on ESPN Radio. game 7 was also the most watched conference finals game on ESPN in four years, averaging 9.9 million viewers and peaking at 12.1 million viewers.

This was the fifth playoff meeting between the two teams, with the Heat winning three of the first four meetings.

Western Conference finals
(3) Golden State Warriors vs. (4) Dallas Mavericks
Stephen Curry had 21 points and a game-high 12 rebounds, while Jordan Poole and Andrew Wiggins both scored 19 as the Warriors took game 1 at home, with Golden State holding Dallas to their lowest field goal percentage of the playoffs (36.0%). Luka Dončić led the Mavericks with 20 points, but shot just 6-of-18 from the field, along with scoring just two points and committing six turnovers in the second half as the Mavericks lost their third straight game 1 of the playoffs.
 Luka Dončić had 18 points in the first quarter, and 42 points total. His 3-pointer with 13 seconds before halftime—the Mavs' 15th of the first half—made it 72–58 at the break. Although they trailed by 19 at one point, Golden State answered with a 25–13 third quarter to pull to 85–83 going into fourth. The Warriors would not look back from there, shooting 78.9% in the fourth quarter as Golden State outscored Dallas 68–45 in the second half. Stephen Curry scored 32 points with six 3-pointers and eight rebounds, while Kevon Looney had a career-high 21 points and 12 rebounds.
 Stephen Curry scored 31 points, while Andrew Wiggins's playoff career-high 27 points helped Golden State beat the Mavericks to take a 3–0 lead in the Western Conference finals. Luka Dončić scored 40 points and shot 4-for-9 from beyond the arc, but that was a blip on an otherwise rough night from 3-point range for the Mavericks, who missed their first seven and finished 13-of-45 (28.9%).
The game is notable for a highlight play that occurred with 6:40 left in the 4th quarter with the score 91–83 in favor of the Warriors. Stephen Curry passed the ball to Andrew Wiggins who drove down the baseline to dunk, posterizing Luka Dončić. The play was controversially called an offensive foul on Wiggins (which would have nullified the dunk), but the call was reversed to no foul and a good basket after a successful challenge by Warriors coach Steve Kerr.

 With the Warriors having a chance to sweep and head to the NBA Finals, the Mavericks dominated the entire game after the 1st quarter, going on a 25–6 run in a span of 7 minutes during the 2nd quarter, eventually leading 62–47 by halftime, and increasing the lead by 29 by the 4th quarter. The Warriors, however, thanks in part to Jonathan Kuminga and Moses Moody, rallied a comeback attempt, decreasing the deficit to 8 with 3:23 left in the 4th quarter, resulting in Steve Kerr to bring back in his starters. However, the Mavericks would silence the rally, keeping their season alive and forcing a fifth game in San Francisco. Luka Dončić scored a near triple-double with 30 points, 14 rebounds and nine assists. Stephen Curry scored 20 points and eight assists, while Jonathan Kuminga grabbed eight rebounds.
 After tearing his left ACL in the 2019 NBA Finals and then tearing his right Achilles' tendon just before the start of the 2020–21 season, Klay Thompson helped send the Warriors to their sixth NBA Finals in the last eight years, scoring 32 points and shooting 8-for-16 from 3-point range. Kevon Looney also scored 10 points and grabbed 18 rebounds, finishing the game with more offensive rebounds (7) than the Mavericks (6).
Stephen Curry was awarded the inaugural Western Conference finals MVP award, averaging 23.8 points, 6.6 rebounds, and 7.4 assists on 44% from the field, 44% from beyond the arc, and 84% from the free throw line.

This was the second playoff meeting between these two teams, with the Warriors winning the first meeting.

NBA Finals: (W3) Golden State Warriors vs. (E2) Boston Celtics
Note: Times are EDT (UTC−4) as listed by the NBA. If the venue is located in a different time zone, the local time is also given.

This was the fifth playoff meeting between the two teams, and the second since the Philadelphia Warriors relocated to the Bay Area in 1962, with the Celtics winning the first four meetings.

Statistical leaders
Media coverage
Television
ESPN, ABC, TNT, and NBA TV broadcast the playoffs nationally in the United States. Regional sports networks affiliated with teams also broadcast first-round games, except for those weekend games televised on ABC.
During the first two rounds, Sunday through Wednesday night games were primarily on TNT, Friday night games on ESPN, and Sunday afternoon games on ABC. Exceptions on those days included a Sunday afternoon game on TNT during the first weekend of the playoffs, ESPN televising one Sunday afternoon game in place of ABC due to F1 coverage, and ABC broadcasting one Friday night first-round game. NBA TV also televised selected Monday through Thursday night games in the first round and served as the overflow of any games televised on TNT (except when games on NBA TV and TNT at the same time in which TBS served as the overflow broadcaster). For Thursday night games, TNT aired them in the first round and ESPN in the second round. Saturday first-round games were spilt, with TNT only airing two games due to no game 7s in that round, ESPN five games, and ABC one game. ABC then aired all Saturday second-round games (except two second-round game 1s on TNT in place of no game 7s in the first round). ESPN/ABC had exclusive coverage of the Eastern Conference finals while TNT had exclusive coverage of the Western Conference finals. ABC had exclusive coverage of the NBA Finals for the 20th straight year.
In Canada, the home market of the Toronto Raptors, national broadcast rights were split approximately equally between the Sportsnet and TSN groups of channels. Under those rights, the two broadcasters were allowed to produce separate Canadian feeds for all games involving the Raptors regardless of round or U.S. broadcaster. Sportsnet and TSN general simulcast the ABC, ESPN, TNT or NBA TV feed for all other series.

Sponsorship
As part of a new multiyear agreement with Google Pixel, the playoffs were officially known as the 2022 NBA Playoffs presented by Google Pixel. which will be the first-ever playoff tournament to be title sponsored.

Notes
See also
List of NBA regular season records

References
External links

Basketball – Reference.com's 2022 Playoffs section
An error occurred: 'NoneType' object is not subscriptable
Bot: I couldn't retrieve any information. Please try again later.
'''

In [53]:
chat_with_openai(str_prompt[:10000])

An error occurred: 'NoneType' object is not subscriptable


# Summarizing the NBA Playoffs datasets

In [61]:
import tiktoken

def split_into_chunks(text, max_tokens=2000, model="gpt-4"):
    tokenizer = tiktoken.encoding_for_model(model)
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk) for chunk in chunks]

def chat_with_openai(messages):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def summarize_large_text(text, max_tokens=2000, model="gpt-4"):
    chunks = split_into_chunks(text, max_tokens=max_tokens, model=model)
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1} of {len(chunks)}...")
        messages = [
            {"role": "system", "content": "You are a helpful assistant that summarizes text."},
            {"role": "user", "content": f"Summarize the following text:\n{chunk}"}
        ]
        summary = chat_with_openai(messages)
        summaries.append(summary)
    return " ".join(summaries)

def final_summarization(summaries, model="gpt-4"):
    messages = [
        {"role": "system", "content": "You are a helpful assistant creating concise summaries."},
        {"role": "user", "content": f"Summarize the following text:\n{summaries}"}
    ]
    return chat_with_openai(messages)

In [62]:
# Example Usage
large_text = "Your very large text here..."
summaries = summarize_large_text(str_prompt)
final_summary = final_summarization(summaries)
print(f"Final Summary:\n{final_summary}")

Processing chunk 1 of 7...
Processing chunk 2 of 7...
Processing chunk 3 of 7...
Processing chunk 4 of 7...
Processing chunk 5 of 7...
Processing chunk 6 of 7...
Processing chunk 7 of 7...
Final Summary:
The 2022 NBA playoffs concluded with the Boston Celtics and the Golden State Warriors advancing to the NBA Finals, marking their first appearances since 2010 and 2015 respectively. The Phoenix Suns and the Miami Heat had entered the playoffs with the best records but the Miami Heat did not advance to the finals. The Denver Nuggets secured the sixth seed spot while the Minnesota Timberwolves were pushed to the play-in tournament. In various series, numerous players stood out including Chris Paul for the Suns, Ja Morant for the Grizzlies, Stephen Curry for the Warriors, and Luka Dončić for the Mavericks. Despite impressive performances from many players, the Celtics and Warriors advanced to the NBA Finals, with Jayson Tatum and Stephen Curry respectively earning the MVP awards. However, 